In [49]:
def mimc_round(x):
    return (x + 1)**3

def iterated_mimc(x, rounds):
    for _ in range(rounds):
        x = mimc_round(x)
    return x

In [50]:
from random import randint, choice

# Step 1: Define the finite field GF(2^5)
F.<a> = GF(2^5)
print("Irreducible polynomial:", F.modulus())

# Step 2: Define the polynomial ring over GF(2) and create a random polynomial
R.<x> = GF(2)[]

#  Random function 
# f = sum(R(randint(0,1)) * x^i for i in range(10))
# two rounds of MiMC
#f = x**9 +x**8 + x**6 + x**4 + x**3 + x**2 + x
# f = x**3
# three rounds of MiMC
f = iterated_mimc(x, 3)+1


print(f)

print("Originl polynomial f in GF(2)[x]:", f)


def int_to_field_poly(i, n = 5):
    # Represent integer i as an element of GF(2^n) using binary expansion
    return sum(((i >> j) & 1) * a**j for j in range(n))

def field_poly_to_int(element, n = 5):
    """
    Convert an element from GF(2^n) into an integer,
    interpreting its polynomial representation (in the generator 'a')
    as a binary number.
    
    Parameters:
      element: an element of GF(2^n)
      n: the degree of the extension (here 5)
    
    Returns:
      An integer corresponding to the binary number formed by the coefficients.
    """
    # Get the polynomial representation of the element.
    # The polynomial is in GF(2)[x] and typically represented in ascending order.
    poly = element.polynomial()
    coeffs = poly.list()  # returns list of coefficients [c0, c1, ..., ck]
    # Interpret these coefficients as binary digits: c0 + c1*2 + c2*2^2 + ...
    result = sum(int(c) * (2**i) for i, c in enumerate(coeffs))
    return result

# Step 3: Build the truth table by evaluating f at each element of F
# Sort the elements of F by their integer representation
sorted_F = sorted(list(F), key=lambda c: field_poly_to_int(c))

# Build a dictionary for the truth table: keys are the inputs (from sorted_F), values are f(input).
truth_table_dict = { x: f(x) for x in sorted_F }

print("Truth table (as dictionary):")
for inp, out in truth_table_dict.items():
    print(inp, "->", out, " (int: %d)" % field_poly_to_int(out))

# Modify one entry in the dictionary: pick an input where the output is not F(1), and change it.
non_one_keys = [x for x, out in truth_table_dict.items() if out != F(1)]
modified_truth_table_dict = truth_table_dict.copy()
if non_one_keys:
    key_to_modify = choice(non_one_keys)
else:
    key_to_modify = sorted_F[0]

old_val = modified_truth_table_dict[key_to_modify]
modified_truth_table_dict[key_to_modify] = F(1)
print("\nModified truth table: Changed entry for", key_to_modify, "from", old_val, "to", F(1))

# Rebuild and print the dictionary (for clarity)
print("\nModified truth table (as dictionary):")
for inp, out in modified_truth_table_dict.items():
    print(inp, "->", out, " (int: %d)" % field_poly_to_int(out))

Irreducible polynomial: x^5 + x^2 + 1
x^27 + x^26 + x^25 + x^19 + x^17 + x^16 + x^12 + x^11 + x^10 + x^4 + x^3
Originl polynomial f in GF(2)[x]: x^27 + x^26 + x^25 + x^19 + x^17 + x^16 + x^12 + x^11 + x^10 + x^4 + x^3
Truth table (as dictionary):
0 -> 0  (int: 0)
1 -> 1  (int: 1)
a -> a^3 + a + 1  (int: 11)
a + 1 -> a + 1  (int: 3)
a^2 -> a^3 + a^2 + a + 1  (int: 15)
a^2 + 1 -> a^2 + 1  (int: 5)
a^2 + a -> a^4  (int: 16)
a^2 + a + 1 -> a^4 + a^3 + 1  (int: 25)
a^3 -> a^3 + a^2 + a  (int: 14)
a^3 + 1 -> a^4 + a^2 + a + 1  (int: 23)
a^3 + a -> a^4 + a^3 + a^2 + a  (int: 30)
a^3 + a + 1 -> a^4 + a^3  (int: 24)
a^3 + a^2 -> a^3 + a^2  (int: 12)
a^3 + a^2 + 1 -> a^4 + a  (int: 18)
a^3 + a^2 + a -> a^4 + a + 1  (int: 19)
a^3 + a^2 + a + 1 -> a^2 + a + 1  (int: 7)
a^4 -> a^4 + a^3 + a^2 + a + 1  (int: 31)
a^4 + 1 -> a^4 + 1  (int: 17)
a^4 + a -> a^4 + a^3 + a^2  (int: 28)
a^4 + a + 1 -> a^3 + a  (int: 10)
a^4 + a^2 -> a^3 + a^2 + 1  (int: 13)
a^4 + a^2 + 1 -> a^2 + a  (int: 6)
a^4 + a^2 + a -

In [51]:
def polynomial_from_truth_table(truth_dict, n=5):
    """
    Given a truth table as a dictionary mapping input elements of GF(2^n) 
    to output elements of GF(2^n) for a function F: GF(2^n) -> GF(2^n),
    returns the unique interpolation polynomial p(x) in GF(2^n)[x] that represents the function.
    
    Parameters:
        truth_dict (dict): A dictionary where each key is an element of GF(2^n) (input)
                           and the corresponding value is F(x) (output).
        n (int): The exponent defining the field GF(2^n). Default is 5.
    
    Returns:
        p (polynomial): The unique polynomial in GF(2^n)[x] that interpolates the truth table.
    """
    # Convert dictionary items into a list of (input, output) pairs.
    points = list(truth_dict.items())
    
    # Define the polynomial ring over F.
    P.<x> = PolynomialRing(F)
    
    # Compute and return the interpolation polynomial.
    poly = P.lagrange_polynomial(points)
    return poly

# Now, find the polynomial that represents the function given by the truth table.
original_poly = polynomial_from_truth_table(truth_table_dict, n=5)
print("\nRecovered interpolation polynomial in GF(2^5)[x]:")
print(original_poly)


recovered_poly = polynomial_from_truth_table(modified_truth_table_dict, n=5)
print("\nRecovered interpolation polynomial for modified function in GF(2^5)[x]:")
print(recovered_poly)


Recovered interpolation polynomial in GF(2^5)[x]:
x^27 + x^26 + x^25 + x^19 + x^17 + x^16 + x^12 + x^11 + x^10 + x^4 + x^3

Recovered interpolation polynomial for modified function in GF(2^5)[x]:
x^31 + x^27 + x^26 + x^25 + x^19 + x^17 + x^16 + x^12 + x^11 + x^10 + x^4 + x^3 + 1


In [52]:
# Step 4: For each input, modify its output and compare the recovered polynomial.
#    if the output equals int_to_field_poly(1) (i.e. '1'), change it to int_to_field_poly(2);
#    otherwise, change it to int_to_field_poly(1).
print("\nComparing polynomials for each single-input modification:")

for key in truth_table_dict:
    # Make a copy of the original truth table dictionary.
    mod_dict = truth_table_dict.copy()
    
    # Modify the output for the current key.
    if mod_dict[key] == int_to_field_poly(1):
        mod_dict[key] = int_to_field_poly(2)
    else:
        mod_dict[key] = int_to_field_poly(1)
    
    # Compute the interpolation polynomial for the modified truth table.
    modified_poly = polynomial_from_truth_table(mod_dict, n=5)
    diff_poly = modified_poly - original_poly
    
    print("\nFor input key:", key)
    print("Original output was:", truth_table_dict[key], " (int:", field_poly_to_int(truth_table_dict[key]), ")")
    print("Modified output is:", mod_dict[key], " (int:", field_poly_to_int(mod_dict[key]), ")")
#     print("\nModified interpolation polynomial:")
#     print(modified_poly)
#     print("\nDifference polynomial (modified - original):")
#     print(diff_poly)
    
    # Now print the coefficients (monomials) for which the original and modified polynomials are identical.
    P.<x> = PolynomialRing(F)  # reinitialize polynomial ring for clarity
    all_exponents = set(original_poly.dict().keys()).union(modified_poly.dict().keys())
    print("\nMonomials with unchanged coefficients:")
    # Convert both polynomials to dictionaries: keys are exponents and values are coefficients.
    orig_dict = original_poly.dict()
    mod_dict = modified_poly.dict()
    all_exponents = set(orig_dict.keys()).union(mod_dict.keys())

    for d in sorted(all_exponents):
        # Get coefficients, defaulting to 0 if the monomial is missing.
        orig_coeff = orig_dict.get(d, 0)
        mod_coeff  = mod_dict.get(d, 0)
        if orig_coeff == mod_coeff:
            print("x^%d: coefficient = %s" % (d, orig_coeff))



Comparing polynomials for each single-input modification:

For input key: 0
Original output was: 0  (int: 0 )
Modified output is: 1  (int: 1 )

Monomials with unchanged coefficients:
x^3: coefficient = 1
x^4: coefficient = 1
x^10: coefficient = 1
x^11: coefficient = 1
x^12: coefficient = 1
x^16: coefficient = 1
x^17: coefficient = 1
x^19: coefficient = 1
x^25: coefficient = 1
x^26: coefficient = 1
x^27: coefficient = 1

For input key: 1
Original output was: 1  (int: 1 )
Modified output is: a  (int: 2 )

Monomials with unchanged coefficients:

For input key: a
Original output was: a^3 + a + 1  (int: 11 )
Modified output is: 1  (int: 1 )

Monomials with unchanged coefficients:

For input key: a + 1
Original output was: a + 1  (int: 3 )
Modified output is: 1  (int: 1 )

Monomials with unchanged coefficients:

For input key: a^2
Original output was: a^3 + a^2 + a + 1  (int: 15 )
Modified output is: 1  (int: 1 )

Monomials with unchanged coefficients:

For input key: a^2 + 1
Original outpu

In [53]:
from sage.crypto.boolean_function import BooleanFunction

# --- Assuming earlier code has defined:
# F, int_to_field_poly, field_poly_to_int, sorted_F, and truth_table_dict
# For example, truth_table_dict maps each input in sorted_F to f(input) in GF(2^5).

print("\nANF of each output coordinate bit:")

n = 5  # number of coordinate bits
for bit in range(n):
    # Build the Boolean function's truth table for the 'bit'-th coordinate.
    # For each input x in sorted_F, extract the bit (using bit-shift and mask).
    tt_bit = [ (field_poly_to_int(truth_table_dict[x]) >> bit) & 1 for x in sorted_F ]
    
    # Create a BooleanFunction from the truth table.
    bf = BooleanFunction(tt_bit)
    
    # Print the ANF for this coordinate function.
    print("\nCoordinate bit", bit, "ANF:")
    print(bf.algebraic_normal_form())



ANF of each output coordinate bit:

Coordinate bit 0 ANF:
x0*x1*x2*x3 + x0*x1*x3*x4 + x0*x1*x4 + x0*x1 + x0*x2 + x0*x4 + x0 + x1*x2*x3*x4 + x1*x2*x3 + x1*x3 + x1 + x2*x3*x4 + x2*x3 + x2*x4 + x2 + x3*x4 + x4

Coordinate bit 1 ANF:
x0*x1*x2*x3 + x0*x1*x2*x4 + x0*x1*x2 + x0*x1*x3*x4 + x0*x1*x3 + x0*x2*x4 + x0*x2 + x0*x4 + x1*x2*x3*x4 + x1*x2*x3 + x1*x3 + x1 + x2 + x3*x4 + x3 + x4

Coordinate bit 2 ANF:
x0*x1*x2*x3 + x0*x1*x2*x4 + x0*x1*x3*x4 + x0*x1*x3 + x0*x2*x3*x4 + x0*x2*x3 + x0*x2*x4 + x0*x3*x4 + x0*x4 + x1*x2*x3*x4 + x1*x2 + x1*x3*x4 + x2*x3*x4 + x2*x3 + x2*x4 + x2 + x3*x4 + x3 + x4

Coordinate bit 3 ANF:
x0*x1*x2*x3 + x0*x1*x2 + x0*x1*x3*x4 + x0*x1 + x0*x2*x3 + x0*x2*x4 + x0*x2 + x0*x3 + x0*x4 + x1*x2*x3 + x1*x2*x4 + x1*x3 + x1*x4 + x1 + x2*x3*x4 + x2*x3 + x2*x4 + x2 + x3 + x4

Coordinate bit 4 ANF:
x0*x1*x2*x3 + x0*x1*x3 + x0*x1*x4 + x0*x2*x3*x4 + x0*x3 + x1*x2*x3 + x1*x2*x4 + x1*x2 + x1*x3*x4 + x1*x3 + x2*x3*x4 + x2*x4 + x4
